# Tutorial 1: Preprocessing Conversational Transcripts

This tutorial demonstrates how to use the ALIGN package to preprocess conversational data for linguistic alignment analysis.

## What You'll Learn

- How to prepare raw conversational transcripts for alignment analysis
- Using different POS taggers (NLTK, spaCy, Stanford)
- Understanding the preprocessing output format
- Validating that outputs are ready for Phase 2 (alignment analysis)

## Prerequisites

You should have already:
1. Cloned this repository
2. Installed the package: `pip install -e .`
3. Sample data is available in `src/align_test/data/CHILDES/`

---
## Step 1: Import and Configure

In [1]:
import os
import pandas as pd
import ast

# Import the preprocessing function
from align_test.prepare_transcripts import prepare_transcripts

print("✓ Imports successful")

/Users/ndd697/.pyenv/versions/3.13.5/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Imports successful


In [4]:
# Configure paths
# Input: Sample CHILDES data (included in the package)
INPUT_DIR = '../src/align_test/data/CHILDES/'

# Output: Where to save preprocessed files
OUTPUT_DIR_BASIC = './tutorial_output/preprocessed_nltk'
OUTPUT_DIR_SPACY = './tutorial_output/preprocessed_spacy'
OUTPUT_DIR_STANFORD = './tutorial_output/preprocessed_stanford'

# Create output directories
for dir_path in [OUTPUT_DIR_BASIC, OUTPUT_DIR_SPACY, OUTPUT_DIR_STANFORD]:
    os.makedirs(dir_path, exist_ok=True)
    
print(f"Input directory: {INPUT_DIR}")
print(f"Output directories created ✓")

Input directory: ../src/align_test/data/CHILDES/
Output directories created ✓


---
## Step 2: Inspect Sample Input Data

Let's look at what raw conversation data looks like before preprocessing.

In [5]:
# Check available files
files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.txt')]
print(f"Found {len(files)} conversation files\n")

# Load and display a sample file
sample_file = os.path.join(INPUT_DIR, files[0])
df_sample = pd.read_csv(sample_file, sep='\t', encoding='utf-8')

print(f"Sample file: {files[0]}")
print(f"Columns: {df_sample.columns.tolist()}")
print(f"Rows: {len(df_sample)}\n")
print("First 5 utterances:")
df_sample.head()

Found 20 conversation files

Sample file: time197-cond1.txt
Columns: ['participant', 'content']
Rows: 127

First 5 utterances:


,participant,content
0,cgv,that was fun.
1,kid,Dad you should have climbed the cliffs with us.
2,cgv,next time I will.
3,kid,did you have fun fishing.
4,cgv,yeah.


---
## Step 3: Basic Preprocessing (NLTK Only)

This is the **fastest option** and requires no additional setup. NLTK tags are automatically downloaded if needed.

### What This Does:
1. Cleans text (removes non-letters, fillers like "um", "uh")
2. Merges adjacent turns by the same speaker
3. Tokenizes and lemmatizes words
4. Adds POS (Part-of-Speech) tags using NLTK

In [6]:
print("Starting NLTK-only preprocessing...\n")

results_nltk = prepare_transcripts(
    input_files=INPUT_DIR,
    output_file_directory=OUTPUT_DIR_BASIC,
    run_spell_check=False,          # Disable for speed (optional)
    minwords=2,                      # Minimum words per utterance
    add_additional_tags=False        # NLTK only
)

print(f"\n✓ Preprocessing complete!")
print(f"Total utterances processed: {len(results_nltk)}")

Starting NLTK-only preprocessing...

  - Downloading wordnet...
    ✓ wordnet downloaded successfully
  - Downloading omw-1.4...
    ✓ omw-1.4 downloaded successfully
NLTK resources ready!


Found 20 files to process
Output directory: ./tutorial_output/preprocessed_nltk

Processing: time197-cond1.txt
  1. Cleaning text...
  2. Merging adjacent turns...
  3. Tokenizing...
  4. Lemmatizing...
  5. Applying POS tagging...
Converting tokens and lemmas to string representations...
Applying NLTK POS tagging...
NLTK POS tagging complete
  6. Saved: time197-cond1.txt
     Rows: 76
Processing: time202-cond1.txt
  1. Cleaning text...
  2. Merging adjacent turns...
  3. Tokenizing...
  4. Lemmatizing...
  5. Applying POS tagging...
Converting tokens and lemmas to string representations...
Applying NLTK POS tagging...
NLTK POS tagging complete
  6. Saved: time202-cond1.txt
     Rows: 92
Processing: time191-cond1.txt
  1. Cleaning text...
  2. Merging adjacent turns...
  3. Tokenizing...
  4. Lemma

### Examine Preprocessed Output

Let's see what the preprocessed data looks like:

In [7]:
print("Output columns:")
for i, col in enumerate(results_nltk.columns, 1):
    print(f"  {i}. {col}")

print(f"\nSample row:")
sample = results_nltk.iloc[0]
print(f"Participant: {sample['participant']}")
print(f"Content: {sample['content']}")
print(f"Tokens: {sample['token']}")
print(f"Lemmas: {sample['lemma']}")
print(f"Tagged: {sample['tagged_token'][:100]}...")

Output columns:
  1. participant
  2. content
  3. token
  4. lemma
  5. tagged_token
  6. tagged_lemma
  7. file

Sample row:
Participant: cgv
Content: that was fun
Tokens: ['that', 'was', 'fun']
Lemmas: ['that', 'be', 'fun']
Tagged: [('that', 'DT'), ('was', 'VBD'), ('fun', 'NN')]...


### Validate Output Format

The preprocessing creates strings that can be parsed back to Python objects. This is important for Phase 2 (alignment analysis).

In [8]:
# Test parsing
sample_row = results_nltk.iloc[0]

# Parse token string back to list
tokens = ast.literal_eval(sample_row['token'])
print(f"Token type after parsing: {type(tokens)}")
print(f"Tokens: {tokens}")

# Parse tagged tokens
tagged = ast.literal_eval(sample_row['tagged_token'])
print(f"\nTagged token type: {type(tagged)}")
print(f"Tagged tokens (first 3): {tagged[:3]}")
print(f"\n✓ Format validation successful!")

Token type after parsing: <class 'list'>
Tokens: ['that', 'was', 'fun']

Tagged token type: <class 'list'>
Tagged tokens (first 3): [('that', 'DT'), ('was', 'VBD'), ('fun', 'NN')]

✓ Format validation successful!


---
## Step 4: Preprocessing with spaCy (Optional)

spaCy provides **100-200x faster** POS tagging than Stanford with minimal accuracy differences.


In [9]:
# Check if spaCy is available
try:
    import spacy
    print("✓ spaCy is installed")
    spacy_available = True
except ImportError:
    print("✗ spaCy not installed")
    print("\nTo install: pip install spacy")
    print("Then run: python -m spacy download en_core_web_sm")
    spacy_available = False

✓ spaCy is installed


In [10]:
if spacy_available:
    print("Starting preprocessing with spaCy...\n")
    
    results_spacy = prepare_transcripts(
        input_files=INPUT_DIR,
        output_file_directory=OUTPUT_DIR_SPACY,
        run_spell_check=False,
        minwords=2,
        add_additional_tags=True,       # Add spaCy tags
        tagger_type='spacy'             # Specify spaCy
    )
    
    print(f"\n✓ spaCy preprocessing complete!")
    
    # Show additional columns
    spacy_cols = [c for c in results_spacy.columns if c not in results_nltk.columns]
    print(f"\nAdditional columns with spaCy:")
    for col in spacy_cols:
        print(f"  - {col}")
else:
    print("⊘ Skipping spaCy preprocessing (not installed)")

Starting preprocessing with spaCy...

  - Downloading wordnet...
    ✓ wordnet downloaded successfully
  - Downloading omw-1.4...
    ✓ omw-1.4 downloaded successfully
NLTK resources ready!


Found 20 files to process
Output directory: ./tutorial_output/preprocessed_spacy

Processing: time197-cond1.txt
  1. Cleaning text...
  2. Merging adjacent turns...
  3. Tokenizing...
  4. Lemmatizing...
  5. Applying POS tagging...
Initializing spaCy POS tagger...
spaCy model 'en_core_web_sm' not found. Downloading...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.6 MB/s eta 0:00:00 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


Successfully downloaded and loaded: en_core_web_sm
spaCy tagger initialized (disabled: ['parser', 'attribute_ruler', 'lemmatizer', 'ner'])
Converting tokens and lemmas to string representations...
Applying NLTK POS tagging...
NLTK POS tagging complete
Applying spaCy POS tagging to 76 utterances...


spaCy tagging lemmas: 100%|██████████| 76/76 [00:00<00:00, 1184.39it/s]

spaCy POS tagging complete
  6. Saved: time197-cond1.txt
     Rows: 76
Processing: time202-cond1.txt
  1. Cleaning text...
  2. Merging adjacent turns...
  3. Tokenizing...
  4. Lemmatizing...
  5. Applying POS tagging...
Initializing spaCy POS tagger...


...

spaCy POS tagging complete
  6. Saved: time201-cond1.txt
     Rows: 90
Processing: time192-cond1.txt
  1. Cleaning text...
  2. Merging adjacent turns...
  3. Tokenizing...
  4. Lemmatizing...
  5. Applying POS tagging...
Initializing spaCy POS tagger...
Loaded spaCy model: en_core_web_sm
spaCy tagger initialized (disabled: ['parser', 'attribute_ruler', 'lemmatizer', 'ner'])
Converting tokens and lemmas to string representations...
Applying NLTK POS tagging...
NLTK POS tagging complete
Applying spaCy POS tagging to 67 utterances...


spaCy tagging lemmas: 100%|██████████| 67/67 [00:00<00:00, 1014.25it/s]


spaCy POS tagging complete
  6. Saved: time192-cond1.txt
     Rows: 67
Processing: time207-cond1.txt
  1. Cleaning text...
  2. Merging adjacent turns...
  3. Tokenizing...
  4. Lemmatizing...
  5. Applying POS tagging...
Initializing spaCy POS tagger...
Loaded spaCy model: en_core_web_sm
spaCy tagger initialized (disabled: ['parser', 'attribute_ruler', 'lemmatizer', 'ner'])
Converting tokens and lemmas to string representations...
Applying NLTK POS tagging...
NLTK POS tagging complete
Applying spaCy POS tagging to 106 utterances...


spaCy tagging lemmas: 100%|██████████| 106/106 [00:00<00:00, 1001.32it/s]

spaCy POS tagging complete
  6. Saved: time207-cond1.txt
     Rows: 106

Saved concatenated dataframe: align_concatenated_dataframe.txt
Total rows: 1832

Processing complete!

Summary:
  - Files processed: 20
  - Total utterances: 1832
  - Output directory: ./tutorial_output/preprocessed_spacy

✓ spaCy preprocessing complete!

Additional columns with spaCy:
  - tagged_spacy_token
  - tagged_spacy_lemma


---
## Step 5: Preprocessing with Stanford (Optional)

Stanford CoreNLP provides the **highest accuracy** but is **~100x slower** than spaCy.

### Prerequisites:

#### 1. Install Java
```bash
# macOS
brew install openjdk

# Linux
sudo apt-get install default-jdk

# Windows
# Download from: https://www.java.com/en/download/
```

#### 2. Download Stanford POS Tagger
- Download from: https://nlp.stanford.edu/software/tagger.shtml#Download
- Extract to a known location (e.g., `~/stanford-postagger/`)
- Update the paths in the cell below

In [ ]:
import subprocess

# Check Java
try:
    result = subprocess.run(['java', '-version'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        version = result.stderr.split('\n')[0]
        print(f"✓ Java installed: {version}")
        java_available = True
    else:
        print("✗ Java not working properly")
        java_available = False
except:
    print("✗ Java not found")
    print("\nPlease install Java first (see instructions above)")
    java_available = False

# Configure STANFORD_PATH (UPDATE THESE FOR YOUR SYSTEM)
if java_available:
    STANFORD_PATH = os.path.expanduser('~/stanford-postagger-full-2020-11-17') # Update this path
    STANFORD_MODEL = 'models/english-left3words-distsim.tagger'
    
    # Check if Stanford tagger exists
    jar_path = os.path.join(STANFORD_PATH, 'stanford-postagger.jar')
    model_path = os.path.join(STANFORD_PATH, STANFORD_MODEL)
    
    if os.path.exists(jar_path) and os.path.exists(model_path):
        print(f"✓ Stanford tagger found at: {STANFORD_PATH}")
        stanford_available = True
    else:
        print(f"✗ Stanford tagger not found at: {STANFORD_PATH}")
        print("\nPlease:")
        print("  1. Download from: https://nlp.stanford.edu/software/tagger.shtml#Download")
        print("  2. Extract and update STANFORD_PATH above")
        stanford_available = False
else:
    stanford_available = False

✓ Java installed: java version "1.8.0_471"
✗ Stanford tagger not found at: /Users/ndd697/stanford-postagger-full-2020-11-17

Please:
  1. Download from: https://nlp.stanford.edu/software/tagger.shtml#Download
  2. Extract and update STANFORD_PATH above


In [ ]:
if stanford_available:
    print("Starting preprocessing with Stanford...")
    print("⚠️  This may take several minutes...\n")
    
    results_stanford = prepare_transcripts(
        input_files=INPUT_DIR,
        output_file_directory=OUTPUT_DIR_STANFORD,
        run_spell_check=False,
        minwords=2,
        add_additional_tags=True,
        tagger_type='stanford',
        stanford_pos_path=STANFORD_PATH,
        stanford_language_path=STANFORD_MODEL,
        stanford_batch_size=50  # Process in batches for speed
    )
    
    print(f"\n✓ Stanford preprocessing complete!")
    
    # Show additional columns
    stanford_cols = [c for c in results_stanford.columns if c not in results_nltk.columns]
    print(f"\nAdditional columns with Stanford:")
    for col in stanford_cols:
        print(f"  - {col}")
else:
    print("⊘ Skipping Stanford preprocessing (not available)")

---
## Summary: Output Files

Your preprocessed files are now ready for alignment analysis!

In [ ]:
print("📁 Preprocessed Files Created:\n")

for label, path in [("NLTK-only", OUTPUT_DIR_BASIC),
                    ("NLTK + spaCy", OUTPUT_DIR_SPACY),
                    ("NLTK + Stanford", OUTPUT_DIR_STANFORD)]:
    if os.path.exists(path):
        files = [f for f in os.listdir(path) if f.endswith('.txt')]
        if files:
            print(f"✓ {label}: {len(files)} files in {path}")

print("\n" + "="*60)
print("✅ TUTORIAL 1 COMPLETE")
print("="*60)
print("\nNext Step: Run Tutorial 2 (Alignment Analysis)")
print("This will analyze the preprocessed files to compute alignment metrics.")

---
## Understanding the Output Format

Each preprocessed file contains:

### Core Columns (always present):
- `participant`: Speaker identifier
- `content`: Original utterance text (cleaned)
- `token`: List of tokenized words (as string)
- `lemma`: List of lemmatized words (as string)
- `tagged_token`: List of (word, POS) tuples from NLTK (as string)
- `tagged_lemma`: List of (lemma, POS) tuples from NLTK (as string)
- `file`: Source filename

### Additional Columns (when using extra taggers):
- `tagged_spacy_token`: POS tags from spaCy
- `tagged_spacy_lemma`: POS tags for lemmas from spaCy
- `tagged_stan_token`: POS tags from Stanford
- `tagged_stan_lemma`: POS tags for lemmas from Stanford

### Important Notes:
- All list/tuple columns are stored as **strings**
- Use `ast.literal_eval()` to convert strings back to Python objects
- This format ensures compatibility with the alignment analysis phase
- A concatenated file with all conversations is also saved for batch processing